# S8 - Fichiers, CSV, JSON et Pandas

Ce notebook couvre :
1. Lecture et écriture de fichiers CSV
2. Transformation CSV → JSON
3. Manipulation de JSON
4. Analyse de données avec pandas
5. Création de rapports

## 1. Importation des Bibliothèques

In [ ]:
import pandas as pd
import json
import csv
from pathlib import Path

print("Bibliothèques importées avec succès!")

## 2. Lecture du Fichier CSV avec Pandas

In [ ]:
# Lire le fichier CSV
df = pd.read_csv('../python_basics/sales_data.csv')

print("Données chargées:")
print(f"Nombre de lignes: {len(df)}")
print(f"Nombre de colonnes: {len(df.columns)}")
print("\nAperçu des données:")
df.head(10)

## 3. Exploration des Données

In [ ]:
# Informations générales
print("=== INFORMATIONS SUR LE DATASET ===")
print(df.info())
print("\n=== STATISTIQUES DESCRIPTIVES ===")
df.describe()

In [ ]:
# Colonnes et types
print("Colonnes du DataFrame:")
for col in df.columns:
    print(f"  - {col}: {df[col].dtype}")

## 4. Nettoyage et Transformation des Données

In [ ]:
# Convertir la date en datetime
df['date'] = pd.to_datetime(df['date'])

# Calculer le montant total
df['montant_total'] = df['quantite'] * df['prix_unitaire']

# Extraire des informations de la date
df['jour_semaine'] = df['date'].dt.day_name()
df['mois'] = df['date'].dt.month

print("Colonnes après transformation:")
print(df.columns.tolist())
print("\nAperçu des nouvelles colonnes:")
df[['date', 'montant_total', 'jour_semaine', 'mois']].head()

## 5. Analyse des Ventes

In [ ]:
# Ventes totales
ventes_totales = df['montant_total'].sum()
print(f"Chiffre d'affaires total: {ventes_totales:,.2f} €")

# Nombre de transactions
nb_transactions = len(df)
print(f"Nombre de transactions: {nb_transactions}")

# Panier moyen
panier_moyen = ventes_totales / nb_transactions
print(f"Panier moyen: {panier_moyen:,.2f} €")

In [ ]:
# Top 5 des produits les plus vendus (quantité)
print("=== TOP 5 PRODUITS (QUANTITÉ) ===")
top_produits_qte = df.groupby('produit')['quantite'].sum().sort_values(ascending=False).head(5)
print(top_produits_qte)

In [ ]:
# Top 5 des produits par chiffre d'affaires
print("=== TOP 5 PRODUITS (CA) ===")
top_produits_ca = df.groupby('produit')['montant_total'].sum().sort_values(ascending=False).head(5)
for produit, ca in top_produits_ca.items():
    print(f"{produit}: {ca:,.2f} €")

In [ ]:
# Ventes par catégorie
print("=== VENTES PAR CATÉGORIE ===")
ventes_categorie = df.groupby('categorie')['montant_total'].sum().sort_values(ascending=False)
for categorie, montant in ventes_categorie.items():
    pourcentage = (montant / ventes_totales) * 100
    print(f"{categorie}: {montant:,.2f} € ({pourcentage:.1f}%)")

In [ ]:
# Meilleurs clients
print("=== TOP 5 CLIENTS ===")
top_clients = df.groupby('client')['montant_total'].sum().sort_values(ascending=False).head(5)
for client, montant in top_clients.items():
    nb_achats = len(df[df['client'] == client])
    print(f"{client}: {montant:,.2f} € ({nb_achats} achats)")

In [ ]:
# Ventes par ville
print("=== VENTES PAR VILLE ===")
ventes_ville = df.groupby('ville')['montant_total'].sum().sort_values(ascending=False)
for ville, montant in ventes_ville.items():
    print(f"{ville}: {montant:,.2f} €")

## 6. Transformation CSV → JSON

In [ ]:
# Convertir le DataFrame en JSON (format records)
json_data = df.to_dict(orient='records')

print("Conversion CSV → JSON réussie!")
print(f"Nombre d'enregistrements: {len(json_data)}")
print("\nAperçu des 2 premiers enregistrements:")
print(json.dumps(json_data[:2], indent=2, default=str))

In [ ]:
# Sauvegarder en JSON
with open('sales_data.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, indent=2, ensure_ascii=False, default=str)

print("✅ Fichier sales_data.json créé avec succès!")

## 7. Lecture et Manipulation du JSON

In [ ]:
# Lire le fichier JSON
with open('sales_data.json', 'r', encoding='utf-8') as f:
    data_from_json = json.load(f)

print(f"Données JSON chargées: {len(data_from_json)} enregistrements")
print("\nPremier enregistrement:")
print(json.dumps(data_from_json[0], indent=2))

In [ ]:
# Reconvertir JSON → DataFrame
df_from_json = pd.DataFrame(data_from_json)

print("DataFrame recréé depuis JSON:")
print(df_from_json.head())
print(f"\nShape: {df_from_json.shape}")

## 8. Création d'un Rapport Résumé

In [ ]:
# Créer un dictionnaire de résumé
resume = {
    "periode": {
        "debut": df['date'].min().strftime('%Y-%m-%d'),
        "fin": df['date'].max().strftime('%Y-%m-%d')
    },
    "statistiques_generales": {
        "nb_transactions": int(len(df)),
        "chiffre_affaires_total": float(df['montant_total'].sum()),
        "panier_moyen": float(df['montant_total'].mean()),
        "quantite_totale": int(df['quantite'].sum())
    },
    "top_produits": top_produits_ca.to_dict(),
    "ventes_par_categorie": ventes_categorie.to_dict(),
    "top_clients": top_clients.to_dict(),
    "ventes_par_ville": ventes_ville.to_dict()
}

# Sauvegarder le résumé
with open('rapport_ventes.json', 'w', encoding='utf-8') as f:
    json.dump(resume, f, indent=2, ensure_ascii=False)

print("✅ Rapport de ventes généré: rapport_ventes.json")
print("\nContenu du rapport:")
print(json.dumps(resume, indent=2, ensure_ascii=False))

## 9. Exercice : Filtrage et Exportation

In [ ]:
# Filtrer les ventes d'informatique supérieures à 100€
ventes_importantes = df[
    (df['categorie'] == 'Informatique') & 
    (df['montant_total'] > 100)
]

print(f"Ventes importantes trouvées: {len(ventes_importantes)}")
print("\nAperçu:")
print(ventes_importantes[['date', 'produit', 'montant_total', 'client']].head(10))

In [ ]:
# Exporter en CSV
ventes_importantes.to_csv('ventes_importantes.csv', index=False, encoding='utf-8')
print("✅ Fichier ventes_importantes.csv créé")

# Exporter en JSON
ventes_importantes.to_json('ventes_importantes.json', orient='records', indent=2)
print("✅ Fichier ventes_importantes.json créé")

## 10. Conclusion

Dans ce notebook, nous avons :
- ✅ Lu et exploré un fichier CSV avec pandas
- ✅ Nettoyé et transformé les données
- ✅ Effectué des analyses statistiques
- ✅ Converti des données entre CSV et JSON
- ✅ Créé des rapports résumés
- ✅ Filtré et exporté des sous-ensembles de données

Ces compétences sont essentielles pour le traitement et l'analyse de données en Python!